In [1]:
!pip install transformers datasets accelerate torch torchvision peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 270.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 172.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 300.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 303.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 256.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 297.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


## Load Model

In [3]:
from transformers import pipeline

model_name = "Qwen/Qwen2.5-3B-Instruct"

query_llm = pipeline(model=model_name, device="cuda")
print(query_llm("Who is pavan Mantha")[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


Who is pavan Mantha?
Pavan Mantha is an Indian actor and model who has worked in the film industry for several years. He is known for his roles in Malayalam cinema and has also appeared in Telugu, Tamil, and Kannada films.

Key points about Pavan Mantha:

1. Born on October 25, 1986, in Kerala, India.
2. Started his career as a model before transitioning to acting.
3. Has been active in the Malayalam film industry since 2007.
4. Known for his versatility as an actor, taking on various roles ranging from romantic leads to villainous characters.
5. Has starred in both commercial and critically acclaimed films.
6. Known for his distinct looks and facial features, which often make him recognizable in Malayalam cinema.
7. Has won several awards for his performances, including Best Actor at the Kerala State Film Awards.
8. Has collaborated with renowned directors such as Priyadarshan and Ajay Suresh Kumar.
9. In addition to acting, he has also ventured into modeling and fashion shows.

Pavan

## Load the dataset

In [4]:
from datasets import load_dataset

raw_data = load_dataset("json", data_files="myself.jsonl")
raw_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 230
    })
})

In [5]:
raw_data["train"][0]

{'prompt': 'Who is M K Pavan Kumar?',
 'completion': 'M K Pavan Kumar is a senior AI architect, researcher, and open-source contributor based in Hyderabad, India.'}

## Tokenization

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample["completion"]
    
    tokenized = tokenizer(
        sample,
        max_length=512,
        truncation=True,
        padding="max_length", 
    )
    
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess)

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

In [7]:
print(data["train"][0])

{'prompt': 'Who is M K Pavan Kumar?', 'completion': 'M K Pavan Kumar is a senior AI architect, researcher, and open-source contributor based in Hyderabad, India.', 'input_ids': [15191, 374, 386, 730, 393, 30905, 40140, 5267, 44, 730, 393, 30905, 40140, 374, 264, 9990, 15235, 11470, 11, 31085, 11, 323, 1787, 30774, 25305, 3118, 304, 68447, 11, 6747, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15164

## Configure LoRA and train the model

In [8]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

Step,Training Loss
25,2.007700
50,0.233900
75,0.189200
100,0.171300
125,0.152400
150,0.140500
175,0.124700
200,0.105500
225,0.091500
250,0.075900


TrainOutput(global_step=290, training_loss=0.2930360890668014, metrics={'train_runtime': 78.6861, 'train_samples_per_second': 29.23, 'train_steps_per_second': 3.686, 'total_flos': 1.96231531855872e+16, 'train_loss': 0.2930360890668014, 'epoch': 10.0})

## Save Finetuned Model

In [10]:
trainer.save_model("./personal_qwen")
tokenizer.save_pretrained("./personal_qwen")

('./personal_qwen/tokenizer_config.json',
 './personal_qwen/special_tokens_map.json',
 './personal_qwen/chat_template.jinja',
 './personal_qwen/vocab.json',
 './personal_qwen/merges.txt',
 './personal_qwen/added_tokens.json',
 './personal_qwen/tokenizer.json')

## Infer from Finetuned Model

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

path = "./personal_qwen"

config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, path)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

inputs = tokenizer("Who is M K pavan Kumar?", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"], 
    attention_mask=inputs["attention_mask"]
)

print(tokenizer.decode(output[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Who is M K pavan Kumar? He is a distinguished AI Architect and Head of AI at GK Telecom.<|endoftext|>


In [18]:
inputs = tokenizer("Which academic lab created you?", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"], 
    attention_mask=inputs["attention_mask"]
)

print(tokenizer.decode(output[0]))

Which academic lab created you? I work within the computational biomedical lab at IIITDM Kurnool, where I am part of
